### Ejercicio 1: Análisis de sentimientos de comentarios de Twitter.

Del siguiente link:

https://www.kaggle.com/code/ngyptr/python-nltk-sentiment-analysis/input

Descargue la información relacionada a comentarios.

1. Revise los valores nulos o incompletos que pueda contener la información de entrada.


In [ ]:
# Espacio para desarrollar el código necesario.








## Ejercicio 2: Conteo de palabras.

Usando la información anterior realice un análisis sobre la frecuencia de palabras, mustre un histograma con el conteo de las palabras )(Recuerde realizar un preprocesamiento a la columna, para que todos las variantes de la misma palabra sean tomadas en cuenta)

In [ ]:
# Espacio para desarrollar el código necesario.









## Ejercicio 3: Análisis de sentimientos.
Usando algunas de las técnicas desarrolladas en clase, desarrolle una algoritmo que permita clasificar los comentarios como: positivos, negativos o neutros, en cada caso genere una nube de palabras.

Agregue comentarios o explicaciones breves en el códogo desarrollado.

In [ ]:
# Espacio para desarrollar el código necesario.









## Ejercicio 4. Clasificadores de documentos.

1. Diseñe un algoritmo para clasificar documentos.
2. Agregue detalles sobre su funcionamiento.
3. Enumere las librerias que podria usar y explique el por qué de su elección.
4. En caso de usar un modelo basado en Inteligencia Artificial explique su funcinamiento.


In [ ]:
# Espacio para desarrollar el código necesario.







